In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
import analysis.lib.purification.purify_ssro as pu_ssro;reload(pu_ssro)
%matplotlib inline
import matplotlib as mpl
base_f = r"M:\tnw\ns\qt\Diamond\Eigenpapers\17_QMemories2\Data\MemoryNOMWErrors"

In [ ]:
### plotting options
mpl.rc("pdf", fonttype =42),
mpl.rc("font",**{'family':'sans-serif','sans-serif':['Helvetica Neue LT Std 55 Roman','DejaVu Sans'],'size':11})# 
mpl.rc("xtick",**{'top':True,'direction':'in'})
mpl.rc("ytick",**{'right':True,'direction':'in'})
mpl.rc("savefig", **{'transparent':True,'bbox':'tight'})
mpl.rc('font', size = 11)

In [ ]:
### additional functions and definitions
def save_plt(fig,name,folder=None):
    if folder == None:
        plt.savefig(os.path.join(base_f, name+'.pdf'),
            format='pdf')
    else:
        plt.savefig(os.path.join(folder, name+'.pdf'),
            format='pdf')
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

In [ ]:
### data extraction
reload(pu_ssro)
tstamp_standard = '183127'
tstamp_pi = '181828'
tstamp_matched  ='190944'
kws = {'folder': base_f,'ret':True,'do_plot':False,'do_fit':True,'do_print':False,'fixed':[0,2,4,5,6]}
fit_pi= pu_ssro.number_of_repetitions(contains = tstamp_pi,**kws)[0]
fit_match = pu_ssro.number_of_repetitions(contains = tstamp_matched,**kws)[0]
fit_standard = pu_ssro.number_of_repetitions(contains = tstamp_standard,**kws)[0]
fit_list = [fit_standard,fit_pi,fit_match]

In [ ]:
### plotting
lab0 = r"$\mathrm{N_{1/e}}=$"
legendlabels = [lab0,lab0,lab0]
default_cols = [  'tab:purple', 'tab:olive',
                'tab:brown', 'tab:pink', 'tab:gray',  'tab:cyan','tab:red','tab:blue', 'tab:orange', 'tab:green']
fig,ax = plt.subplots(figsize=(4,3))
for cur_fit,lab,col in zip(fit_list,legendlabels,default_cols):
    fit_x = np.linspace(np.amin(cur_fit['x']),480,100)
    current_legend = lab + str(np.int(cur_fit['params_dict']['T'])) + r' $\pm$ '+ str(np.int(cur_fit['error_dict']['T']))
    ax.errorbar(cur_fit['x'],cur_fit['y'],cur_fit['y_u'],fmt='o',label = current_legend,color = col)
    ax.plot(fit_x,cur_fit['fitfunc'](fit_x),'-',color=col)
    
plt.legend(frameon=False)
plt.xlim([0,500])
plt.ylabel('Equatorial Bloch vector length')
plt.xlabel('Entangling attempts')

### save to server
# save_plt(fig,'Fig4_Mem_decay')
### save to local repo
# save_plt(fig,'Fig4_Mem_decay',folder = r'D:\nkalb\Desktop\Purification\NoCode2\memories_v2\Analysis')

## analyze sweep time

In [ ]:
### more func definitions
def plot_fit_vals(list_of_fits,vals):
    fig,ax = plt.subplots()
    for val in vals:
        for fits in list_of_fits:
            val_res,val_res_u,val_x = [],[],[] ## init empty
            for f in fits:
                if f['error_dict'][val] > 20000:
                    print 'had to exclude one result. Errorbar VERY LARGE'
                else:
                    val_x.append(f['a'].g.attrs['LDE_decouple_time']/2.256e-6)
                    val_res.append(f['params_dict'][val]);val_res_u.append(f['error_dict'][val])
            ax.errorbar(val_x,val_res,val_res_u,label = val)
    plt.xlabel(r'decoupling time ($\tau_L$)')
    plt.ylabel(val)
    plt.legend()
    return fig,ax

def plot_all_raw(fits):
    fig,ax = plt.subplots()
    for fit in fits:
        fit_x = np.linspace(fit['x'][0],fit['x'][-1],100)
        order = np.round(fit['a'].g.attrs['LDE_decouple_time']/2.256e-6,2)
        ax.errorbar(fit['x'],fit['y'],fit['y_u'],fmt='o',label=r't = %s * $\tau_L$' % order,zorder=5)
        ax.plot(fit_x,fit['fitfunc'](fit_x),color='black',zorder=0)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xlabel(fit['a'].g.attrs['sweep_name'])

In [ ]:
## data selection and analysis style
base_f2 = r"M:\tnw\ns\qt\Diamond\Eigenpapers\17_QMemories2\Data\NoMWErrorsSweepTime"
kws = {'folder': base_f2,'ret':True,'do_plot':False,'do_fit':True,'show_guess':False,
       'do_print':False,'fixed':[0,2,4,5,6],'return_all':True}

# tstampsxC61 = ['151038','151644','152248','152853','153456','154101','154709',
#            '155318_','155928','160539']
# tstampszC61 = ['151419','152023','152627','153231','153834','154441','155051',
#            '155701','160311','160922']
# tstampsxC3 = ['202222','203002','203736','204457','205230','210003','210727','211459','212229','213005']
# tstampszC3 = ['202717','203451','204214','204944','205718','210443','211214','211944','212716','213459']#,'','']

### C6
kwsC6_x = merge_two_dicts(kws,{'contains':'C6_X','older_than':'20171102_232823','newer_than':'20171102_223352',})
kwsC6_x_100nW = merge_two_dicts(kws,{'contains':'C6_0.1uW_X','older_than':'20171103_231238','newer_than':'20171103_041337',})
kwsC6_z_100nW = merge_two_dicts(kwsC6_x_100nW,{'contains':'C6_0.1uW_Z'});kwsC6_z_100nW['fit_A'] = -0.8;kwsC6_z_100nW['fit_T']=100
C6_x_1uW=merge_two_dicts(kwsC6_x_100nW,{'contains':'C6_1.0uW_X',})
C6_x_4uW=merge_two_dicts(kwsC6_x_100nW,{'contains':'C6_4.0uW_X',})


### C3
kwsC3_x_100nW = merge_two_dicts(kws,{'contains':'C3_0.1uW_X','older_than':'20171103_190025','newer_than':'20171103_012211',})
C3_x_1uW=merge_two_dicts(kwsC3_x_100nW,{'contains':'C3_1.0uW_X',})
C3_x_4uW=merge_two_dicts(kwsC3_x_100nW,{'contains':'C3_4.0uW_X',})

# some massaging of z for C3
kwsC3_z_100nW = merge_two_dicts(kwsC3_x_100nW,{'contains':'C3_0.1uW_Z'})
kwsC3_z_100nW['fit_A'] = -0.9;kwsC3_z_100nW['fit_T']=3000;


### C1
kwsC1_x_100nW = merge_two_dicts(kws,{'contains':'C1_0.1uW_X','older_than':'20171103_012000','newer_than':'20171102_234400',})
kwsC1_z_100nW = merge_two_dicts(kwsC1_x_100nW,{'contains':'C1_0.1uW_Z'});kwsC1_z_100nW['fit_A'] = -0.8;kwsC1_z_100nW['fit_T']=100


In [ ]:
reload(pu_ssro)
#gather data
data_set = C1_x_1uW
fits = pu_ssro.number_of_repetitions(**kwsC3_x_100nW)
fits2 = pu_ssro.number_of_repetitions(**C3_x_1uW)
fits3 = pu_ssro.number_of_repetitions(**C3_x_4uW)
list_of_fits=[fits,fits2,fits3]

In [ ]:
i = 6
f = fits3[i:i+2]
plot_all_raw(f)
print f[0]['params_dict']['T'],f[0]['error_dict']['T']
fig,ax = plot_fit_vals(list_of_fits,['T'])
plt.show()
plt.close('all')